In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
import sys
sys.path.append('../scripts')

In [3]:
import os
# Disable weights and biases (if installed)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
os.environ["WANDB_DISABLED"] = "true"

In [4]:
from pathlib import Path

from dataset import load_data, get_dataloader
from generative.run_experiment import get_training_args, get_trainer, get_tokenizer

/home/Florian.Borchert/miniconda3/envs/ellipses/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Training

In [17]:
import hydra
from hydra import compose, initialize

hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(config_path=Path('..'), job_name='foo', version_base='1.1')
config = compose(config_name='experiment.yaml')

In [18]:
training_args = get_training_args(config, report_to="none")
tokenizer = get_tokenizer(config)

PyTorch: setting up devices
loading configuration file config.json from cache at /home/Florian.Borchert/.cache/huggingface/hub/models--google--mt5-base/snapshots/d86816880b5acc27e697e52bc237e816dc828b17/config.json
Model config MT5Config {
  "_name_or_path": "google/mt5-base",
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "transformers_version": "4.23.1",
  "use_cache": true,
  "vocab_size": 2

In [19]:
base_path = Path('..')
train_df, val_df, test_df = load_data(base_path / config.data.cnf_tsv_path, base_path / config.data.controls_tsv_path)

In [20]:
# Ellipses
(~train_df.controls).sum(), (~val_df.controls).sum(), (~test_df.controls).sum()

(2250, 510, 485)

In [21]:
# Controls
(train_df.controls).sum(), (val_df.controls).sum(), (test_df.controls).sum()

(2208, 521, 516)

In [22]:
train_dataset, val_dataset, test_dataset = get_dataloader(train_df, val_df, test_df, tokenizer)

In [15]:
training_args.num_train_epochs = 10

In [16]:
trainer = get_trainer(config, tokenizer, training_args, train_dataset, val_dataset)

In [15]:
trainer.train()

/home/Florian.Borchert/miniconda3/envs/ellipses/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4510
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1128
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Exact Match,Google Bleu
1,0.665600,0.252594,0.191419,0.694391
2,0.278900,0.116620,0.589659,0.741167


***** Running Evaluation *****
  Num examples = 909
  Batch size = 8
***** Running Evaluation *****
  Num examples = 909
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1128, training_loss=2.5238845356812716, metrics={'train_runtime': 662.3681, 'train_samples_per_second': 13.618, 'train_steps_per_second': 1.703, 'total_flos': 1888722478596096.0, 'train_loss': 2.5238845356812716, 'epoch': 2.0})

In [ ]:
eval_metrics = trainer.evaluate(val_dataset)
print(eval_metrics)

test_metrics = trainer.evaluate(test_dataset, metric_key_prefix='test')
print(test_metrics)

# Error Analysis

In [16]:
from transformers import Text2TextGenerationPipeline
from evaluation import error_analysis, Metrics

In [17]:
pipeline = Text2TextGenerationPipeline(model=trainer.model, tokenizer=tokenizer, max_length=config.generation_max_length, device=0)

In [18]:
from evaluation import error_analysis, get_scores

In [19]:
import pandas as pd

def calculate_errors(out, sample):
    gen_text = [o['generated_text'] for o in out]
    errors = error_analysis(gen_text, sample.full_resolution, sample.raw_sentence)
    display(pd.concat([errors.error_type.value_counts(), errors.error_type.value_counts() / len(errors)], axis=1))    
    return errors

In [20]:
%%time
out_valid = pipeline(list(val_df.raw_sentence))

CPU times: user 7min 56s, sys: 161 ms, total: 7min 56s
Wall time: 7min 56s


In [ ]:
errors_valid = calculate_errors(out_valid, val_df)

In [ ]:
%%time
scores = get_scores(errors_valid, "eval")
scores

In [ ]:
%%time
out_test = pipeline(list(test_df.raw_sentence))

In [ ]:
errors_test = calculate_errors(out_test, test_cnfs)

In [ ]:
%%time
scores = get_scores(errors_test, "test")
scores